In [66]:
import pandas as pd

import seaborn as sns
pal = sns.color_palette()

import plotly.express as px
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objects as go
import openpyxl
import plotly.io as pio
pio.renderers.default = "vscode"

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')

from IPython.display import display
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings

 # FILE READING, DATA CLEANING AND ANALYSIS ON PRICING

In [67]:
# Import dataset
print("Loading dataset...")
# df = pd.read_csv("./assets/get_around_pricing_project.csv")  # sep = ";"
pricing_df = pd.read_csv('assets/get_around_pricing_project.csv')
print("...Done.")
pd.set_option('display.max_columns', None)

Loading dataset...
...Done.


In [68]:
# Basic stats
print("Number of rows : {}".format(pricing_df.shape[0]))
print("Number of columns : {}".format(pricing_df.shape[1]))
print()

print("Display of dataset: ")
display(pricing_df.head())
print()

print("Basics statistics: ")
data_desc = pricing_df.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")

pd.concat([pricing_df.isnull().sum(), 100 * pricing_df.isnull().sum()/len(pricing_df)], axis=1).rename(columns={0:'Missing Records', 1:'Percentage (%)'})


Number of rows : 4843
Number of columns : 15

Display of dataset: 


,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183



Basics statistics: 


,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
count,4843.000000,4843,4.843000e+03,4843.00000,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843.000000
unique,NaN,28,NaN,NaN,4,10,8,2,2,2,2,2,2,2,NaN
top,NaN,Citroën,NaN,NaN,diesel,black,estate,True,True,False,False,False,False,True,NaN
freq,NaN,969,NaN,NaN,4641,1633,1606,2662,3839,3865,3881,2613,3674,4514,NaN
mean,2421.000000,NaN,1.409628e+05,128.98823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121.214536
std,1398.198007,NaN,6.019674e+04,38.99336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.568268
min,0.000000,NaN,-6.400000e+01,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000
25%,1210.500000,NaN,1.029135e+05,100.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.000000
50%,2421.000000,NaN,1.410800e+05,120.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.000000
75%,3631.500000,NaN,1.751955e+05,135.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.000000



Percentage of missing values: 


,Missing Records,Percentage (%)
Unnamed: 0,0,0.0
model_key,0,0.0
mileage,0,0.0
engine_power,0,0.0
fuel,0,0.0
paint_color,0,0.0
car_type,0,0.0
private_parking_available,0,0.0
has_gps,0,0.0
has_air_conditioning,0,0.0


Nous constatons que nous premierement que nous n'avons pas de valeurs manquantes, ce qui est une tres bonne chose. Par contre nous constatons que nous avons des outliers , notamment le kilometrage minimum qui est de -64 km, ce qui est impossible.

In [69]:
pricing_df[pricing_df['mileage'] < 0]

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
2938,2938,Renault,-64,230,diesel,black,sedan,True,True,False,True,False,False,True,274


Vu qu'on a qu'un seul élément dans ce cas, nous allons supprimer ce outlier.

In [70]:
pricing_df = pricing_df[pricing_df['mileage'] >= 0]

print("Basics statistics: ")
data_desc = pricing_df.describe(include='all')
display(data_desc)

Basics statistics: 


,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
count,4842.000000,4842,4.842000e+03,4842.000000,4842,4842,4842,4842,4842,4842,4842,4842,4842,4842,4842.000000
unique,NaN,28,NaN,NaN,4,10,8,2,2,2,2,2,2,2,NaN
top,NaN,Citroën,NaN,NaN,diesel,black,estate,True,True,False,False,False,False,True,NaN
freq,NaN,969,NaN,NaN,4640,1632,1606,2661,3838,3864,3881,2612,3673,4513,NaN
mean,2420.893226,NaN,1.409919e+05,128.967369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121.182982
std,1398.322665,NaN,6.016882e+04,38.970348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.499826
min,0.000000,NaN,4.760000e+02,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000
25%,1210.250000,NaN,1.029658e+05,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.000000
50%,2420.500000,NaN,1.410845e+05,120.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.000000
75%,3631.750000,NaN,1.752062e+05,135.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.000000


In [71]:
# droping useless features

pricing_df = pricing_df.drop(['Unnamed: 0'], axis=1)
pricing_df.head(1)


,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106


In [72]:
pricing_df['model_key'].value_counts()

model_key
Citroën        969
Renault        915
BMW            827
Peugeot        642
Audi           526
Nissan         275
Mitsubishi     231
Mercedes        97
Volkswagen      65
Toyota          53
SEAT            46
Subaru          44
Opel            33
Ferrari         33
PGO             33
Maserati        18
Suzuki           8
Porsche          6
Ford             5
KIA Motors       3
Alfa Romeo       3
Fiat             2
Lexus            2
Lamborghini      2
Mini             1
Mazda            1
Honda            1
Yamaha           1
Name: count, dtype: int64

# EDA

In [73]:
# plotting cars by model

# colors = sns.color_palette("colorblind", len(years))
colors = pal

df_model = pricing_df['model_key'].value_counts()[:10]
label = df_model.index
size = df_model.values

trace = go.Pie(labels=label, values=size, marker=dict(colors=colors),hole = .2)

data = [trace]
layout = go.Layout(
    title='Percentage of Ten First Cars by Model'
)

fig = go.Figure(data=data, layout=layout)


py.iplot(fig)

On constate que Citroên est la marque la plus présente dans la flotte, suivie de Renault.

In [74]:
# plotting rental price par day

df_price = pricing_df.rental_price_per_day.value_counts()

trace = go.Bar(
    x=df_price.index,
    y=df_price.values,
    marker=dict(
        color = df_price.values,
        colorscale='Jet',
        showscale=True)
)

data = [trace]
layout = go.Layout(xaxis=dict(tickangle=15),
    title='Rental Price Per Day', 
    yaxis = dict(title = 'Number of observations'))

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

On constate que les prix des locations tournent beaucoup entre 90€ et 140€ par jour, en fonction du model et caractéristiques de la voiture. la voiture la plus chère coute 422€ par jour et la moins chère coute 10€.

In [75]:
df_mean_price = pricing_df.groupby('model_key')['rental_price_per_day'].mean().reset_index()
df_mean_price = df_mean_price.sort_values(by='rental_price_per_day', ascending=False)

trace = go.Bar(
    x=df_mean_price['model_key'],
    y=df_mean_price['rental_price_per_day'],
    marker=dict(
        color=df_mean_price['rental_price_per_day'],
        colorscale='Jet',
        showscale=True)
)

data = [trace]
layout = go.Layout(xaxis=dict(tickangle=15),
                   title='Average rental price per day by car model',
                   yaxis=dict(title='Average rental price per day'))

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)


On constate qu'en moyenne,les voitures de marques Suzuki sont les plus chères à la location avec un prix d'environ $224/jour tandis que les Mazda sont les moins chères avec un prix d'environ $67/jour.

In [76]:
df_sum_price = pricing_df.groupby('model_key')['rental_price_per_day'].sum().reset_index()
df_sum_price = df_sum_price.sort_values(by='rental_price_per_day', ascending=False)

trace = go.Bar(
    x=df_sum_price['model_key'],
    y=df_sum_price['rental_price_per_day'],
    marker=dict(
        color=df_sum_price['rental_price_per_day'],
        colorscale='Jet',
        showscale=True)
)

data = [trace]
layout = go.Layout(xaxis=dict(tickangle=15),
                   title='Sum of rental price by car model on year',
                   yaxis=dict(title='Sum of rental price'))

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

On constate que les models de voiture Renault sont les plus rentables à la location sur l'année

In [77]:
# Ploting percentage of cars by model an fuel type
cont_table = pd.crosstab(pricing_df["model_key"], pricing_df["fuel"], normalize="index")


fig = px.bar(cont_table, x=cont_table.index, y=cont_table.columns, barmode='group')


fig.update_layout(
    title="Percentage Of Cars by Model and Fuel",
    xaxis_title="Car's Model",
    yaxis_title="Percentage",
    yaxis_tickformat = '.2%',
    legend_title="Fuel"
)

fig.show()

On constate que le diesel est de loin le type de carburant le plus utilsé et que les voitures de marque Porsche dans la flotte sont à 50% Hybride-Essence et 50% electrique, pas de moteur thermique.

In [78]:
pricing_df.head()

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


In [79]:
# Correlation Matrix


correlation = pricing_df[['rental_price_per_day', 'mileage', 'engine_power',  'private_parking_available', 'has_gps', 'has_air_conditioning', 'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']].corr()
cols = correlation.columns.values
corr  = correlation.values

# Création d'un Heatmap avec les données
trace = go.Heatmap(z=corr,
                   x=cols,
                   y=cols,
                   reversescale=True)

# Ajout des annotations de texte à l'intérieur du Heatmap
annotations = []
for i in range(corr.shape[0]):
    for j in range(corr.shape[1]):
        annotations.append(dict(x=cols[j], y=cols[i], text='{:.2f}'.format(corr[i, j]), font=dict(color='white'), showarrow=False))

# Configuration du layout
layout = go.Layout(dict(title="Correlation Matrix for features",
                        autosize=False,
                        height=600,
                        width=1200,
                        margin=dict(l=200),
                        yaxis=dict(tickfont=dict(size=8)),
                        xaxis=dict(tickfont=dict(size=8)),
                        annotations=annotations))

# Création de la figure avec le Heatmap et le layout
fig = go.Figure(data=[trace], layout=layout)

# Affichage de la figure
fig.show()


Les headmap nous montre que le moins de kilometrage possible, la puissant du moteur, le type boite de vitesse (manuel ou auto) et les autres options de confort contribuent au prix plus élevé de la location d'un véhicule.